In [1]:
!pip install docling langchain langchain-community langchain-huggingface faiss-cpu

In [2]:
!pip install -qU langchain-ollama


In [3]:
import os
from typing import List
from langchain_ollama import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter  # For text chunking
from langchain_huggingface import HuggingFaceEmbeddings  # For text embeddings
from langchain_community.vectorstores import FAISS  # Vector database
from langchain.chains import RetrievalQA  # For question-answering pipeline
from langchain.prompts import PromptTemplate  # For customizing LLM prompts

In [4]:
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = True

doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
            )
        }
    )

In [6]:
from docling_core.types.doc import DoclingDocument
def extract_pdf_content(converter: DocumentConverter ,file_path) -> DoclingDocument:
    """
    Extract structured content from PDF using Docling library
    
    Args:
        file_path (str): Path to the PDF file
        
    Returns:
        str: Extracted content in markdown format
    """
    result = converter.convert(file_path)
    return result.document


file_path = "Input-context.pdf"
document = extract_pdf_content(doc_converter,file_path)

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/84/16/8416a7eb6bc0964a8abb5bb890afca2b8384fdc1e010a788e6c411a97c4d2305/31e60b4709571b613bc8736a9c982fb550d8d7a1809160a68a8282af60c8910b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1744877810&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDg3NzgxMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzg0LzE2Lzg0MTZhN2ViNmJjMDk2NGE4YWJiNWJiODkwYWZjYTJiODM4NGZkYzFlMDEwYTc4OGU2YzQxMWE5N2M0ZDIzMDUvMzFlNjBiNDcwOTU3MWI2MTNiYzg3MzZhOWM5ODJmYjU1MGQ4ZDdhMTgwOTE2MGE2OGE4MjgyYWY2MGM4OTEwYj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=bevWOWBvjIfmIm1G26Cv0sjSaQK%7EyNpXicf8tD-PiJTvqGoUhrqfo8QFbpZaOia7Ucy4xhaVck2e4oSDC%7EPPLeRYfgJkZ--kBl52Q2Mhbs2AzuMFy%7Ej95oR8xEyNUJx93SCURjGqFZXoJe4jYhz5ETup1y75pHeKuWFVPzg5JA2FoOrbB19G5JcMiYtGzk-5r-ezYmpuu2nWZ%7ElZsG7dk6c3wjYh2r4oDWdh5%7EiNhx1dPNkQ72BK-LJaFy

KeyboardInterrupt: 

In [10]:
def saveMarkdown(result):
    with open("extracted.md", "w",  encoding="utf-8") as mdFile:
        mdFile.write(result)


def save_tables_to_csv(tables, base_filename="table"):
    """
    Save a list of pandas DataFrames to CSV files in a specified directory.

    Args:
        tables (List[pd.DataFrame]): List of pandas DataFrames.
        output_dir (str or Path): Folder where CSVs should be saved.
        base_filename (str): Base name for each CSV file.
    """
    output_dir = "./Tables"
    try:
        os.makedirs(output_dir, exist_ok= False)
    except FileExistsError:
        print("Files already exists will not override!")
        return

    for i, df in enumerate(tables):
        filename = f"{output_dir}/{base_filename}_{i+1}.csv"
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Saved table {i+1} to {filename}")

saveMarkdown(document.export_to_markdown())
tables = [table.export_to_dataframe() for table in document.tables]
save_tables_to_csv(tables)

NameError: name 'document' is not defined

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000,  # Maximum chunk size in characters
        chunk_overlap=600,  # Overlap between chunks to maintain context
        is_separator_regex=False
    )

with open("extracted.md", "r",  encoding="utf-8") as mdFile:
    structured_content = mdFile.read()

text_chunks = text_splitter.split_text(structured_content)


In [8]:
from langchain.document_loaders import CSVLoader
folder_path = './Tables'
files = os.listdir(folder_path)
all_documents = []

# Loop through each file and load documents
for csv_file in files:
    loader = CSVLoader(f"{folder_path}/{csv_file}")
    documents = loader.load()
    all_documents.extend(documents)  # Append the loaded documents to the list

RuntimeError: Error loading ./Tables/table_15.csv

In [12]:
def create_vector_store(texts: List[str]) -> FAISS:
    """
    Create and initialize FAISS vector store using text embeddings
    
    Args:
        texts (List[str]): List of text chunks to be embedded
        
    Returns:
        FAISS: Initialized FAISS vector store containing embedded texts
    """
    # Initialize sentence transformer model for text embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L12-v2"
    )
    vector_store = FAISS.from_texts(texts, embeddings)
    return vector_store

vector_store = create_vector_store(text_chunks)
vector_store.add_documents(all_documents)

['8a495a3b-9897-438b-b504-97f03ecb0451',
 '14996091-aebe-42be-b4fa-f3d88ea48caa',
 '9f025555-5eca-4494-9c7f-e3376beb1418',
 '39a4962e-8319-465f-8fe0-5057e0ce8019',
 '4cecde45-8e2b-4072-9ea8-b276541a80b3',
 'd98fc07b-9034-44d5-b458-51ea26c4ea2d',
 '0b27929e-6ade-4a8e-9ebe-8b29f5cf4ebb',
 '4af633fb-f43a-49c9-86a4-69839f07f092',
 'c2e87052-3d3a-4b66-b7c9-2a9bba1ad862',
 '53133405-2503-4826-bd82-44822c9746a2',
 '60f4e11f-5fb9-4988-9bab-278af459bc2d',
 '5e40d007-3736-4283-addd-11ee21e46ccd',
 '4301585d-335f-4e8e-9e01-c656917b7da2',
 '7f5fe022-7f9e-4f42-a121-2ccf51db3dff',
 'de634eea-97a9-49ef-900b-ed26ceed1965',
 '5d868ada-f27d-4038-af1f-f0ca9ff3db0c',
 '8064d057-b247-405a-a0d6-5bf8c72b6ba4',
 '033d4ffe-82b3-4415-a8e6-ee9dd2ab3bd0',
 'ee0a715e-4504-4e74-86fe-3a8388d39889',
 '8b3832fa-5c76-4ce7-b1cd-c2c7518d3117',
 'f5998d3f-4cc9-4f13-9d82-c90c344d0983',
 'e072292b-4c2d-4559-871b-9f2329223d5e',
 'c66470f6-b1f1-41f0-af32-4c16ce375384',
 '7751e179-20ef-418a-97f7-384a4a79445d',
 '9b1666d4-5385-

In [13]:
from uuid import uuid4
from langchain_core.chat_history import InMemoryChatMessageHistory

session_id = str(uuid4())  # Or a user/session ID from your app
store = {}  # Simple in-memory store; replace with a persistent one for production

def get_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [14]:
from langchain_core.runnables.history import RunnableWithMessageHistory

def get_chain(vector_store):
    """
    Create question-answering chain using LLM and vector store
    
    Args:
        vector_store (FAISS): Vector store containing embedded documents
        
    Returns:
        RetrievalQA: Configured QA chain for answering questions
    """
    # Initialize local LLM using Ollama
    llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0
    )
    
    # Define custom prompt template for better QA responses
    prompt_template = """
        Use the following pieces of context to answer the question at the end.
        Check context very carefully and reference and try to make sense of that before responding.
        If you don't know the answer, just say you don't know.
        Don't try to make up an answer.
        Answer must be as detailed as possible.
        Think step-by-step.
        If a summary table is available, output it.
        Do not include image placeholders in your response.

        Context: {context}
        
        Question: {question}
        
        Answer:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    # Configure QA chain with retrieval and prompt settings
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Combines all retrieved docs into single context
        retriever=vector_store.as_retriever(search_kwargs={"k": 5}),  # Retrieve top 3 relevant chunks
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True,  # Include source documents in response
    )

    qa_chain_with_history = RunnableWithMessageHistory(
        qa_chain,
        get_session_history=get_history,
        input_messages_key="query",  # default RetrievalQA input key
        history_messages_key="history",
    )

    return qa_chain_with_history

chain = get_chain(vector_store)

In [16]:
import httpx

try:
    with httpx.Client(timeout=10) as client:
        response = client.get("http://your-api-endpoint.com")
except httpx.RequestError as e:
    print(f"Error: {e}")


In [17]:
question = "walk me through configuring the contacts tab when creating a carrier profile"
print(f"\nQuestion: {question}")
response = chain.invoke({"query": question},
                        config={"configurable": {"session_id": session_id}})
print(f"\nAnswer: {response['result']}")


Question: walk me through configuring the contacts tab when creating a carrier profile


ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it

2025-04-17 10:57:19.343 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-17 10:57:19.684 
  command:

    streamlit run c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-17 10:57:19.685 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [14]:
!ollama pull llama3.2:latest #run this to download llama3.2 in the environment so you can use it. 2GB

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Error: could not connect to ollama app, is it running?


In [18]:
import gradio as gr
# Define the function to get the response from the LLM
def get_response(question, history):
    # Simulating the call to chain.invoke (make sure to replace this with your actual implementation)
    response = chain.invoke({"query": question},
                            config={"configurable": {"session_id": session_id}})
    return response['result']

# Use gr.Blocks with a Monochrome theme
with gr.Blocks(theme=gr.themes.Soft(), fill_height=True, fill_width=True) as demo:
        # Textbox for the response output
        answer_output = gr.ChatInterface(fn=get_response,title="Vanrise Assistant", type="messages", editable=True, save_history=True)

# Launch the Gradio interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Error in RootListenersTracer.on_chain_end callback: KeyError('output')
Error in RootListenersTracer.on_chain_end callback: KeyError('output')
